In [ ]:
# Inicializacoes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import datetime as dt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as datetime
import plotly.express as px
import time
import tensorflow as tf
import math
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
from sklearn import preprocessing
from joblib import dump, load

url_dataset='https://github.com/rafasilveira/ibovespa-stocks-prediction/blob/master/data/b3_stocks_1994_2020.csv?raw=true'
ticker='PETR4'

In [ ]:
# carga de dados para uma acao

brute = pd.read_csv(url_dataset)
data = brute[brute["ticker"] == ticker]
# data['dt'] = pd.to_datetime(data.datetime.values)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5640 entries, 209606 to 1881973
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datetime  5640 non-null   object 
 1   ticker    5640 non-null   object 
 2   open      5640 non-null   float64
 3   close     5640 non-null   float64
 4   high      5640 non-null   float64
 5   low       5640 non-null   float64
 6   volume    5640 non-null   float64
dtypes: float64(5), object(2)
memory usage: 352.5+ KB


In [ ]:
# grafico
df_work = data.copy()
df_work.drop(columns=['ticker'],inplace=True)

# plt.figure(figsize=(15,7))
# sns.set()
# sns.lineplot(x = df_work.datetime, y = 'close',data=df_work).set_title(f'Fechamento para {ticker}')

In [ ]:
# treino e teste
X = df_work[['open', 'close', 'high', 'low', 'volume']]
Y = df_work[['close']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle = False)


In [ ]:
# treinamento
# grid search
# n arvores: 10 - 1000
# min amostras divisao: 2 - 10
# max nós folha: 2 - 5
# profundidade maxima: 1 - 50

crossvalidation = KFold(n_splits=10,shuffle=False)

estimator = RandomForestRegressor()
hyperparams = {
    'n_estimators': [10, 20, 100],
    'max_depth': [None, 25, 50],
    'min_samples_split': [2, 5],
    'max_leaf_nodes': [None, 2, 5],
}

grid = GridSearchCV(estimator, param_grid=hyperparams, scoring='neg_mean_squared_error', verbose=1, return_train_score=False, cv=crossvalidation, n_jobs=-1)
grid.fit(X_train, np.ravel(Y_train))

Fitting 10 folds for each of 54 candidates, totalling 540 fits


GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [None, 25, 50],
                         'max_leaf_nodes': [None, 2, 5],
                         'min_samples_split': [2, 5],
                         'n_estimators': [10, 20, 100]},
             scoring='neg_mean_squared_error', verbose=1)

In [ ]:
print(f"score da melhor: {abs(grid.best_score_)}")
print(grid.best_params_)

score da melhor: 725.7934062106243
{'max_depth': 25, 'max_leaf_nodes': None, 'min_samples_split': 2, 'n_estimators': 10}


In [ ]:
# Predicao
y_pred = grid.best_estimator_.predict(X_test)
y_pred = y_pred.reshape(-1, 1)

combined_array = np.concatenate((Y_test,y_pred),axis=1)
df_final = pd.DataFrame(data = combined_array, columns=["actual", "predicted"])
mse = mean_squared_error(Y_test, y_pred)
rmse = math.sqrt(mse)
print(f"Test score - MSE: {mse}, RMSE: {rmse}")

Test score - MSE: 0.0011377269503546103, RMSE: 0.0337302082761819


In [ ]:
# preparar dados para plotagem
a = np.repeat(1, len(Y_test))
b = np.repeat(2, len(y_pred))

df1 = pd.DataFrame(data = np.concatenate((Y_test,(np.reshape(a, (-1, 1)))),axis=1), columns=["price","type",])
df2 = pd.DataFrame(data = np.concatenate((y_pred,(np.reshape(b, (-1, 1)))),axis=1), columns=["price","type"])

frames = [df1, df2]
result = pd.concat(frames, ignore_index=False)

result["type"].replace({1: "actual", 2: "predict"}, inplace=True)

In [ ]:
fig = px.line(result, x=result.index.values, y="price", color='type')
fig.show()